# DATA CARD

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

## Query - Week

In [3]:
w10 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,1)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

w12 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,9)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

w15 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,12)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

w16 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,14)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

w18 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,20)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

w19 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,16)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

w20 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,23)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

w21 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,25)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

w22 = pd.read_sql_query("DECLARE @week INT, @year INT SET datefirst 1\
    SET @week=datepart(week, GetDate()) SET @year=datepart(year, GetDate())\
    \
    select description Description, d1 Monday, d2 Tuesday, d3 Wednesday, d4 Thursday, d5 Friday\
    from KPower_BI.RHL_DataCard_Report(@year,@week,26)\
    where itemsId in (4, 5, 8, 72, 73)\
    order by itemsId", cnxn)

In [4]:
# Add warehouse number column
w10['Warehouse'] = 10
w12['Warehouse'] = 12
w15['Warehouse'] = 15
w16['Warehouse'] = 16
w18['Warehouse'] = 18
w19['Warehouse'] = 19
w20['Warehouse'] = 20
w21['Warehouse'] = 21
w22['Warehouse'] = 22

In [5]:
# Adding Year to the OUTBOUND ORDERS row
w10.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w10.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

w12.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w12.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

w15.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w15.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

w16.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w16.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

w18.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w18.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

w19.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w19.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

w20.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w20.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

w21.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w21.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

w22.at[0, 'Description'] = 'OUTBOUND ORDERS 2021'
w22.at[1, 'Description'] = 'OUTBOUND ORDERS 2022'

In [6]:
# Concatenate all warehouses
df = pd.concat([w10, w12, w15, w16, w18, w19, w20, w21, w22])

In [7]:
# Change the order of the columns. Warehouse first
df = df[['Warehouse', 'Description', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']]
df.head(16)

,Warehouse,Description,Monday,Tuesday,Wednesday,Thursday,Friday
0,10,OUTBOUND ORDERS 2021,1079,668,791,808,430
1,10,OUTBOUND ORDERS 2022,534,0,0,0,0
2,10,NEW ORDERS,541,0,0,0,0
3,10,CARRYOVER ORDERS,653,0,0,0,0
4,10,OPEN ORDERS > 7 DAYS,5,0,0,0,0
0,12,OUTBOUND ORDERS 2021,46,43,18,46,20
1,12,OUTBOUND ORDERS 2022,53,0,0,0,0
2,12,NEW ORDERS,20,0,0,0,0
3,12,CARRYOVER ORDERS,0,0,0,0,0
4,12,OPEN ORDERS > 7 DAYS,3,0,0,0,0


In [8]:
# Save DF into json file
df.to_json('data/data_card.json', orient='records')

In [9]:
# Close connection
cnxn.close()

In [10]:
# Save Waves DF into json file
#df2.to_json('data/card.json')

# Save Waves DF into json file
#df2.to_json('data/card_records.json', orient='records')

# Save Waves DF into json file
#df2.to_json('data/card_split.json', orient='split')

# Save Waves DF into json file
#df2.to_json('data/card_index.json', orient='index')

# Save Waves DF into json file
#df2.to_json('data/card_columns.json', orient='columns')

# Save Waves DF into json file
#df2.to_json('data/card_values.json', orient='values')

# Save Waves DF into json file
#df2.to_json('data/card_table.json', orient='table')